# Obtenção de rating do contribuinte

Com o objetivo de direcionar o modelo que ditá quais dívidas são melhores de recuperar, será realizado um rating que envolva apenas as variáveis do contribuinte.

In [41]:
import os
import dotenv
import zipfile
import pandas as pd
import numpy as np
from datetime import date

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go

import pickle
import shutil

In [42]:
import warnings
warnings.filterwarnings("ignore")

In [43]:
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

True

In [44]:
import locale

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

# 01) Importando dados

In [45]:
zip_file = os.path.join(dataPath, 'rating_igr_20_09.zip')
z = zipfile.ZipFile(zip_file)

In [46]:
def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df

In [47]:
base_imovel = ler_bases_exportadas('imovel.csv')
base_mercantil = ler_bases_exportadas('mercantil.csv')
base_notas_fiscais = ler_bases_exportadas('emissao_notas.csv')

In [111]:
df = base_imovel[base_imovel['cda'] == '0000331f601a73e52b46f673bf0c61251']
df

,cda,tipo_divida,da_aberto,deb_totais,valor_tot,deb_pagos,valor_pago,vlr_tributo,vlr_taxa,inscricao_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,cpf_cnpj_existe,protesto,divida_ajuizada,edificacao,anos_idade_da,quantidade_reparcelamento
1,0000331f601a73e52b46f673bf0c61251,imovel,0,6.0,0.0,0.0,0.0,147.2,196.92,2012-10-17,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,1,0,0,1,11.0,0


In [48]:
# VALOR DE IMÓVEL EM ABERTO
result_imovel = base_imovel[base_imovel['da_aberto'] == 1]
result_imovel = formatar_moeda(result_imovel['valor_tot'].sum())
result_imovel

'R$ 5.884.819.985,56'

In [49]:
# VALOR DE MERCANTIL EM ABERTO
result_mercantil = base_mercantil[base_mercantil['da_aberto'] == 1]
result_mercantil = formatar_moeda(result_mercantil['valor_tot'].sum())
result_mercantil

'R$ 8.722.209.058,04'

In [50]:
base_mercantil.columns

Index(['cda', 'tipo_divida', 'da_aberto', 'deb_totais', 'valor_tot',
       'deb_pagos', 'valor_pago', 'vlr_tributo', 'vlr_taxa',
       'inscricao_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'cpf_cnpj_existe', 'protesto', 'divida_ajuizada',
       'edificacao', 'anos_idade_da', 'quantidade_reparcelamento'],
      dtype='object')

In [51]:
contagem_cda = base_imovel['cda'].value_counts()
registros_repetidos = contagem_cda[contagem_cda > 1]
df_registros_repetidos = registros_repetidos.reset_index()
df_registros_repetidos.columns = ['cda', 'contagem']
df_registros_repetidos

,cda,contagem


In [11]:
cda_teste = base_imovel[base_imovel['cda'] == '661dbac7686f85476b3a8e268c80aedf1']
cda_teste

,cda,tipo_divida,da_aberto,deb_totais,valor_tot,deb_pagos,valor_pago,vlr_tributo,vlr_taxa,inscricao_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,cpf_cnpj_existe,protesto,divida_ajuizada,edificacao,anos_idade_da,quantidade_reparcelamento
516776,661dbac7686f85476b3a8e268c80aedf1,imovel,1,3.0,2038.07,2.0,218.3,2256.37,0.0,2020-01-10,e85d7a978a1b0a9e,CASA,ATIVO,IPTU,1,0,0,1,3.0,0


In [52]:
check = pd.concat([base_imovel, base_mercantil])

In [54]:
# VALOR DE MERCANTIL QUE ESTA EM ABERTO
result_total = check[check['da_aberto'] == 1]
result_total = formatar_moeda(result_total['valor_tot'].sum())
result_total

'R$ 14.607.029.043,60'

---

In [55]:
base_conjunta = pd.concat([base_imovel, base_mercantil])

In [56]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'da_aberto', 'deb_totais', 'valor_tot',
       'deb_pagos', 'valor_pago', 'vlr_tributo', 'vlr_taxa',
       'inscricao_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'cpf_cnpj_existe', 'protesto', 'divida_ajuizada',
       'edificacao', 'anos_idade_da', 'quantidade_reparcelamento'],
      dtype='object')

In [57]:
base_conjunta['data_divida'] = pd.to_datetime(base_conjunta['inscricao_divida'], infer_datetime_format = True)
base_conjunta['ano_inscricao_da'] = base_conjunta['data_divida'].dt.year

In [58]:
#status_situacao0 = base_imovel[base_imovel['da_aberto'] == 1]
status_situacao0 = base_conjunta[base_conjunta['da_aberto'] == 1]
status_situacao0 = status_situacao0[status_situacao0['anos_idade_da'] < 15]
status_situacao0 = status_situacao0[status_situacao0['cpf_cnpj_existe'] == 0]
status_situacao0 = formatar_moeda(status_situacao0['valor_tot'].sum())
status_situacao0

'R$ 478.880.439,14'

# 02) Criando variáveis para clusterização

In [59]:
# incorporei a variáve tipo_divida que antes não tinha

dados_pessoas = base_conjunta[['cda', 'id_pessoa', 'situacao', 'cpf_cnpj_existe', 'edificacao', 'deb_totais', 'deb_pagos', 'valor_tot', 'valor_pago', 'tipo_divida']]
dados_pessoas['id_pessoa'] = dados_pessoas['id_pessoa'].astype(str)  # Convertendo para string

dados_pessoas.dropna(subset=['id_pessoa'], inplace=True)


In [78]:
# # Renomeio a coluna criada anteriormente para valor_pago
# dados_pessoas.rename(columns={'valor_pago_vista_parc':'valor_pago'}, inplace=True)

In [60]:
# Calculo que apresenta quantas cdas o contribuinte tem

frequencia_da_pessoa = dados_pessoas.groupby(['id_pessoa'])['cda'].nunique()
total_debitos_pessoa = dados_pessoas.groupby(['id_pessoa'])['deb_totais'].sum()
debitos_pagos_pessoa = dados_pessoas.groupby(['id_pessoa'])['deb_pagos'].sum()
valor_total_pessoa = dados_pessoas.groupby(['id_pessoa'])['valor_tot'].sum()
valor_pago_pessoa = dados_pessoas.groupby(['id_pessoa'])['valor_pago'].sum()

In [61]:
# Agrega informação da base de notas fiscais
dados_pessoas = pd.merge(
    left=dados_pessoas, left_on='id_pessoa', 
    right=base_notas_fiscais, right_on='id_pessoa', how='left'
)

In [62]:
# Verifica valores zerados

dados_pessoas.isnull().sum()

cda                      0
id_pessoa                0
situacao                 0
cpf_cnpj_existe          0
edificacao               0
deb_totais               0
deb_pagos                0
valor_tot                0
valor_pago               0
tipo_divida              0
qtd_notas_2anos    2346979
dtype: int64

In [63]:
# Substituindo por zero os valores nulos
# dados_pessoas['edificacao'] = dados_pessoas['edificacao'].fillna(0)
dados_pessoas['qtd_notas_2anos'] = dados_pessoas['qtd_notas_2anos'].fillna(0)

In [64]:
# Cria variável de situação do contribuinte tratando mercantil e imovel em suas respectivas variáveis ( DEBATER COM EQUIPE )

# MERCANTIL
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] > 0) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 2
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] > 0) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 1
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] == 0) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 1
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] == 0) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 0

# IMOVEL
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 1), 'situacao_ativa'] = 2
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0), 'situacao_ativa'] = 1
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0) & (dados_pessoas['cpf_cnpj_existe'] == 0), 'situacao_ativa'] = 0

# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 1) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 2
# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 1
# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 1) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 0
# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 0

# o que é nulo colocamos peso 0 
dados_pessoas['situacao_ativa'] = dados_pessoas['situacao_ativa'].fillna(0)

In [ ]:
# status_situacao0 = dados_pessoas[dados_pessoas['situacao_ativa'] == 0]
# status_situacao0

In [22]:
# contagem_situacao = status_situacao0['situacao'].value_counts()
# contagem_situacao

INAPTO      711848
SUSPENSO    145524
BAIXADO      32470
Name: situacao, dtype: int64

In [23]:
# contagem_tipo_divida = status_situacao0['tipo_divida'].value_counts()
# contagem_tipo_divida

mercantil    889842
Name: tipo_divida, dtype: int64

In [65]:
# Cria variável com peso sobre a situação real tratando cpf

dados_pessoas['status_situacao'] = dados_pessoas['situacao_ativa'] + dados_pessoas['cpf_cnpj_existe'] 
dados_pessoas.loc[dados_pessoas['situacao_ativa'] == 0, 'status_situacao'] = 0
# dados_pessoas.loc[(dados_pessoas['cpf_cnpj_existe'] == 0 ), 'status_situacao'] = 0

In [ ]:
dados_pessoas

---

In [66]:
# Remove duplicatas dos dados finalizando o dataframe
dados_pessoas.drop_duplicates(subset=['id_pessoa'], inplace=True)
dados_pessoas = dados_pessoas.set_index('id_pessoa')

dados_pessoas['frequencia_da_pessoa'] = frequencia_da_pessoa
dados_pessoas['total_debitos_pessoa'] = total_debitos_pessoa
dados_pessoas['debitos_pagos_pessoa'] = debitos_pagos_pessoa
dados_pessoas['valor_total_pessoa'] = valor_total_pessoa
dados_pessoas['valor_pago_pessoa'] = valor_pago_pessoa

In [67]:
# Faz o calculo do historico de pagamento

dados_pessoas.loc[(dados_pessoas['total_debitos_pessoa'].isna()) | (dados_pessoas['total_debitos_pessoa'] == 0) , 'total_debitos_pessoa'] = 1
dados_pessoas.loc[(dados_pessoas['valor_total_pessoa'].isna()) | (dados_pessoas['valor_total_pessoa'] == 0) , 'valor_total_pessoa'] = 1

dados_pessoas['historico_pagamento_em_qtd'] = dados_pessoas['debitos_pagos_pessoa'] / (dados_pessoas['total_debitos_pessoa'])
dados_pessoas['historico_pagamento_em_valor'] = dados_pessoas['valor_pago_pessoa'] / (dados_pessoas['valor_total_pessoa'])

In [72]:
# Monta dataframe para clusterização

df_pipe_cluster = dados_pessoas.query("frequencia_da_pessoa > 1")
df_pipe_cluster = df_pipe_cluster[['status_situacao', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]

In [73]:
df_pipe_cluster.loc[df_pipe_cluster['historico_pagamento_em_valor'] > 1, 'historico_pagamento_em_valor'] = 1

In [74]:
df_pipe_cluster.head()

,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor
id_pessoa,,,
ac28642d7c82b33f,3.0,0.000000,0.000000
870c08c252b25ad1,3.0,0.622120,1.000000
d8b23eda9800b9e3,2.0,0.017699,0.007503
9f3bac8718dac1fa,3.0,0.524123,1.000000
0bff42777c16d00c,3.0,0.000000,0.000000


In [75]:
df_pipe_cluster.describe().T

,count,mean,std,min,25%,50%,75%,max
status_situacao,352303.0,1.694425,1.370388,0.0,0.0,2.0,3.000000,3.0
historico_pagamento_em_qtd,352303.0,0.253483,0.355775,0.0,0.0,0.0,0.491848,1.0
historico_pagamento_em_valor,352303.0,0.292062,0.424889,0.0,0.0,0.0,0.817820,1.0


# 03) Clusterização dos contribuintes

In [76]:
faixa_n_clusters = [i for i in range(2,16)]
valores_inercia = []
valores_score = []

for k in faixa_n_clusters:
    agrupador = KMeans(n_clusters=k, random_state=1337)
    label = agrupador.fit_predict(df_pipe_cluster)
    print(f"Treinamento do agrupador para K= {k} finalizado")
    
    media_inercia = agrupador.inertia_
    valores_inercia.append(media_inercia)
    print(f"Inércia calculada para o agrupador de K= {k}. Inércia: {media_inercia}")

    media_score = agrupador.score(df_pipe_cluster)
    valores_score.append(media_score)
    print(f"Score calculado para o agrupador de K= {k}. Socre: {media_score}")

Treinamento do agrupador para K= 2 finalizado
Inércia calculada para o agrupador de K= 2. Inércia: 134032.21022719314
Score calculado para o agrupador de K= 2. Socre: -134032.21022719314
Treinamento do agrupador para K= 3 finalizado
Inércia calculada para o agrupador de K= 3. Inércia: 66523.26077403652
Score calculado para o agrupador de K= 3. Socre: -66523.26077403652
Treinamento do agrupador para K= 4 finalizado
Inércia calculada para o agrupador de K= 4. Inércia: 36103.049979940995
Score calculado para o agrupador de K= 4. Socre: -36103.049979941
Treinamento do agrupador para K= 5 finalizado
Inércia calculada para o agrupador de K= 5. Inércia: 25306.643766638135
Score calculado para o agrupador de K= 5. Socre: -25306.64376663814
Treinamento do agrupador para K= 6 finalizado
Inércia calculada para o agrupador de K= 6. Inércia: 15840.303603680202
Score calculado para o agrupador de K= 6. Socre: -15840.3036036802
Treinamento do agrupador para K= 7 finalizado
Inércia calculada para o ag

In [77]:
fig =go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_clusters, y = valores_inercia))
fig.update_layout(
    title="INDICADOR: Inercia para K grupos",
    xaxis_title="Valores de K",
    yaxis_title="Inércia",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [78]:
fig =go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_clusters, y = valores_score))
fig.update_layout(
    title="Indicador: Erro quadratico médio para K grupos",
    xaxis_title="Valores de K",
    yaxis_title="MSE",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [79]:
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 15, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [80]:
valor_ideal_k = optimal_number_of_clusters(valores_inercia)

print("Melhor valor de K:", valor_ideal_k)

Melhor valor de K: 5


In [81]:
# Construindo o melhor agrupador de clusteres
valor_de_k = 4

agrupador = KMeans(n_clusters=valor_de_k, random_state=1337)
agrupador.fit_transform(df_pipe_cluster)

# Obtendo o ponto central dos clusteres
centros = agrupador.cluster_centers_
df_centroide = pd.DataFrame(centros, columns = df_pipe_cluster.columns).round(3)
df_centroide['cluster'] = df_centroide.index

# Obtendo o label para cada pessoa
df_pipe_cluster['label_cluster'] = agrupador.labels_

In [82]:
df_centroide

,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,cluster
0,3.000,0.120,0.088,0
1,0.006,0.076,0.069,1
2,2.893,0.766,0.968,2
3,2.000,0.077,0.076,3


Na totalidade temos +1 cluster especial que é do contribuinte que apareceu a primiera vez em DA

In [83]:
dicionario_clusteres ={
    4: 'PRIMEIRA DIVIDA',
    1: 'PIOR PAGADOR',
    3: 'PAGADOR INTERMEDIARIO',
    0: 'BOM PAGADOR',
    2: 'MELHOR PAGADOR'
}

In [84]:
df_cluster = df_pipe_cluster.groupby('label_cluster')['label_cluster'].value_counts().to_frame()
total = df_cluster['label_cluster'].sum()
df_cluster['perc'] = df_cluster['label_cluster']/total
df_cluster

,,label_cluster,perc
label_cluster,label_cluster,,
0,0,83868,0.238056
1,1,134231,0.381010
2,2,85263,0.242016
3,3,48941,0.138917


Classificações dos contribuintes com base no melhor ao pior pagador, baseado no seu histórico e na sua situação atual.

# 04) Cria classificador de class do contribuinte

In [85]:
# Constroi o modelo que prevê qual o grupo do contribuinte

x_cluster = df_pipe_cluster.drop(columns=['label_cluster'])
y_cluster = df_pipe_cluster['label_cluster']

X_train, X_test, y_train, y_test = train_test_split(x_cluster, y_cluster, test_size=0.3, random_state=1337)

In [86]:
model_predict_contribuinte = RandomForestClassifier(random_state=1337)
model_predict_contribuinte.fit(X_train, y_train)

score_validacao = model_predict_contribuinte.score(X_test, y_test)
print("Score de validacao:", score_validacao)

Score de validacao: 0.9997256152368698


In [87]:
# Previsão da classificação para a base total de contribuintes

matriz_previsao_class = dados_pessoas[['status_situacao', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]
dados_pessoas['class_contribuinte'] = model_predict_contribuinte.predict(matriz_previsao_class)
dados_pessoas.loc[dados_pessoas['frequencia_da_pessoa'] == 1, 'class_contribuinte'] = 4


In [88]:
primeira_divida = dados_pessoas[dados_pessoas['frequencia_da_pessoa'] == 1]

# Como ficou a nossa classificação originalmente

df_primeira_divida = primeira_divida.groupby('class_contribuinte')['class_contribuinte'].value_counts().to_frame()#.reset_index()
df_primeira_divida.rename(columns = {'class_contribuinte':'num_contrib'}, inplace = True)
total = df_primeira_divida['num_contrib'].sum()
df_primeira_divida['perc'] = np.round(df_primeira_divida['num_contrib']/total, 3)
df_primeira_divida

,,num_contrib,perc
class_contribuinte,class_contribuinte,,
4,4,72729,1.0


In [89]:

# Como ficou a nossa classificação originalmente

df_descritiva = dados_pessoas.groupby('class_contribuinte')['class_contribuinte'].value_counts().to_frame()#.reset_index()
df_descritiva.rename(columns = {'class_contribuinte':'num_contrib'}, inplace = True)
total = df_descritiva['num_contrib'].sum()
df_descritiva['perc'] = np.round(df_descritiva['num_contrib']/total, 3)
df_descritiva

,,num_contrib,perc
class_contribuinte,class_contribuinte,,
0,0,83877,0.197
1,1,134233,0.316
2,2,85254,0.201
3,3,48939,0.115
4,4,72729,0.171


In [90]:
# Nomeando a classificação com label de prioridade
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 4, 'class_contribuinte_nome'] = 'PRIMEIRA DIVIDA'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 1, 'class_contribuinte_nome'] = 'PIOR PAGADOR'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 3, 'class_contribuinte_nome'] = 'PAGADOR INTERMEDIARIO'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 0, 'class_contribuinte_nome'] = 'BOM PAGADOR'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 2, 'class_contribuinte_nome'] = 'MELHOR PAGADOR'

In [91]:
df_classificao_contribuinte = dados_pessoas[['class_contribuinte_nome']].reset_index()

In [97]:
df_classificao_contribuinte

,id_pessoa,class_contribuinte_nome
0,ac28642d7c82b33f,BOM PAGADOR
1,870c08c252b25ad1,MELHOR PAGADOR
2,d8b23eda9800b9e3,PAGADOR INTERMEDIARIO
3,9f3bac8718dac1fa,MELHOR PAGADOR
4,0bff42777c16d00c,BOM PAGADOR
...,...,...
425027,aba0af9479986a18,PRIMEIRA DIVIDA
425028,49bc65f01166e4e7,PRIMEIRA DIVIDA
425029,bf79480d4e9ebf0e,PRIMEIRA DIVIDA
425030,391c352ffdfe4e93,PRIMEIRA DIVIDA


In [99]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'da_aberto', 'deb_totais', 'valor_tot',
       'deb_pagos', 'valor_pago', 'vlr_tributo', 'vlr_taxa',
       'inscricao_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'cpf_cnpj_existe', 'protesto', 'divida_ajuizada',
       'edificacao', 'anos_idade_da', 'quantidade_reparcelamento',
       'data_divida', 'ano_inscricao_da', 'percentual_pago_cda'],
      dtype='object')

# 05) Análise discriminante da classificação do contribuintes para o % Pago das dívidas de cada CDA

In [92]:
# Calcula variável target y
base_conjunta['percentual_pago_cda'] = base_conjunta['valor_pago'] / base_conjunta['valor_tot']

In [ ]:
# Substitui por zero os valores nulos em que a divisão for zero
base_conjunta['percentual_pago_cda'] = base_conjunta['percentual_pago_cda'].fillna(0)

In [106]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'da_aberto', 'deb_totais', 'valor_tot',
       'deb_pagos', 'valor_pago', 'vlr_tributo', 'vlr_taxa',
       'inscricao_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'cpf_cnpj_existe', 'protesto', 'divida_ajuizada',
       'edificacao', 'anos_idade_da', 'quantidade_reparcelamento',
       'data_divida', 'ano_inscricao_da', 'percentual_pago_cda'],
      dtype='object')

In [110]:
df = base_conjunta[base_conjunta['percentual_pago_cda'].isna()]
df[['cda', 'tipo_divida', 'valor_tot', 'valor_pago', 'vlr_tributo', 'vlr_taxa', 'anos_idade_da', 'ano_inscricao_da', 'percentual_pago_cda']]

,cda,tipo_divida,valor_tot,valor_pago,vlr_tributo,vlr_taxa,anos_idade_da,ano_inscricao_da,percentual_pago_cda
1,0000331f601a73e52b46f673bf0c61251,imovel,0.0,0.0,147.20,196.92,11.0,2012,NaN
5,0000521b64bf28c8e52281a70553db461,imovel,0.0,0.0,245.50,245.33,7.0,2016,NaN
6,000057f68f7b77276b3a8e268c80aedf1,imovel,0.0,0.0,555.67,592.39,5.0,2018,NaN
8,0000681a0944cefb6b3a8e268c80aedf1,imovel,0.0,0.0,264.24,249.24,4.0,2019,NaN
10,00007af008add2c7e52281a70553db461,imovel,0.0,0.0,745.57,184.90,9.0,2014,NaN
...,...,...,...,...,...,...,...,...,...
1188172,febe110bfbc654739ceed47545e621dd2,mercantil,0.0,0.0,604.45,0.00,21.0,2002,NaN
1188173,bff0d52848d50c739989bae6f4af91ee2,mercantil,0.0,0.0,0.00,2983.66,9.0,2014,NaN
1188174,f370a631c8957ce86b3a8e268c80aedf2,mercantil,0.0,0.0,0.00,1833.42,11.0,2012,NaN
1188175,cb6508b33f8cfd86d8a3e5cebc255ca62,mercantil,0.0,0.0,0.00,2983.66,9.0,2014,NaN


In [116]:
df2 = base_conjunta[base_conjunta['percentual_pago_cda'] == np.inf]
df2[['cda', 'tipo_divida', 'valor_tot', 'valor_pago', 'vlr_tributo', 'vlr_taxa', 'anos_idade_da', 'ano_inscricao_da', 'percentual_pago_cda']]
# df2 = formatar_moeda(df2['vlr_tributo'].sum())
df2

,cda,tipo_divida,da_aberto,deb_totais,valor_tot,deb_pagos,valor_pago,vlr_tributo,vlr_taxa,inscricao_divida,...,tipo_tributo,cpf_cnpj_existe,protesto,divida_ajuizada,edificacao,anos_idade_da,quantidade_reparcelamento,data_divida,ano_inscricao_da,percentual_pago_cda
3,000040eda866e3d19ceed47545e621dd1,imovel,0,4.0,0.0,3.0,5438.40,10779.87,0.00,2015-01-30,...,IPTU,1,0,1,1,8.0,2,2015-01-30,2015,inf
9,00006f113b5e15229ceed47545e621dd1,imovel,0,110.0,0.0,50.0,3522.19,11929.83,372.43,2015-06-30,...,IPTU,1,0,1,1,8.0,1,2015-06-30,2015,inf
12,0000905d980177f36b3a8e268c80aedf1,imovel,0,14.0,0.0,14.0,528.49,272.20,256.29,2020-12-31,...,IPTU,1,1,0,1,3.0,0,2020-12-31,2020,inf
16,0000ea1f35374e602b46f673bf0c61251,imovel,0,12.0,0.0,12.0,423.24,241.56,181.68,2017-10-21,...,IPTU,1,0,0,1,6.0,0,2017-10-21,2017,inf
18,0000f7ef8630f89c9ceed47545e621dd1,imovel,0,35.0,0.0,34.0,2161.46,7154.44,0.00,2008-10-30,...,IPTU,1,0,1,1,15.0,1,2008-10-30,2008,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188099,d2f8b2880b29a956e52281a70553db46E,mercantil,0,13.0,0.0,12.0,5229.52,10498.82,0.00,2022-08-07,...,ISS,1,0,1,0,1.0,1,2022-08-07,2022,inf
1188124,8b2ff1092bf27d6318565e26094f06ee2,mercantil,0,33.0,0.0,2.0,570.84,14863.62,0.00,2025-12-25,...,ISS,1,1,0,0,-2.0,1,2025-12-25,2025,inf
1188127,8373db41370473909989bae6f4af91ee2,mercantil,0,2.0,0.0,2.0,990.88,0.00,990.88,2016-05-14,...,ISS,1,0,0,0,7.0,0,2016-05-14,2016,inf
1188137,d5c15aec74f8551de52281a70553db462,mercantil,0,6.0,0.0,6.0,310.75,310.75,0.00,2011-05-25,...,ISS,1,0,0,0,12.0,1,2011-05-25,2011,inf


In [93]:
# Separação dos dados para a análise discriminante dos grupos de contribuintes

df_analise_discriminante = pd.merge(
    left=base_conjunta, left_on='id_pessoa', right=df_classificao_contribuinte, right_on='id_pessoa', how='left'
)

df_analise_discriminante = df_analise_discriminante[['percentual_pago_cda', 'class_contribuinte_nome']]

In [94]:
df_analise_discriminante.groupby("class_contribuinte_nome").describe().T

class_contribuinte_nome     BOM PAGADOR  MELHOR PAGADOR  \
percentual_pago_cda count  4.633300e+05        377291.0   
                    mean            inf             inf   
                    std             NaN             NaN   
                    min    0.000000e+00             0.0   
                    25%    0.000000e+00             NaN   
                    50%    0.000000e+00             NaN   
                    75%    7.086523e-02             NaN   
                    max             inf             inf   

class_contribuinte_nome    PAGADOR INTERMEDIARIO  PIOR PAGADOR  \
percentual_pago_cda count               227857.0      327429.0   
                    mean                     inf           inf   
                    std                      NaN           NaN   
                    min                      0.0           0.0   
                    25%                      0.0           0.0   
                    50%                      0.0           0.0   
                    75%                      0.0           0.0   
                    max                      inf           inf   

class_contribuinte_nome    PRIMEIRA DIVIDA  
percentual_pago_cda count          49098.0  
                    mean               inf  
                    std                NaN  
                    min                0.0  
                    25%                0.0  
                    50%                NaN  
                    75%                NaN  
                    max                inf

In [95]:
# Dummyzando a variável de classificação 

ohe = OneHotEncoder(dtype=int)

colunas_ohe = ohe.fit_transform(df_analise_discriminante[['class_contribuinte_nome']]).toarray()
df_2 = pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['class_contribuinte_nome']))

df_n_categorico = df_analise_discriminante.drop(columns=['class_contribuinte_nome'], axis=1)
df_pipe_discriminante = pd.concat([df_n_categorico, df_2], axis=1)

In [96]:
x_analise_discriminante = df_pipe_discriminante.drop(columns=['percentual_pago_cda'])
y_analise_discriminante = df_pipe_discriminante['percentual_pago_cda'].astype('int')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [143]:
analise_discriminante = LinearDiscriminantAnalysis()
analise_discriminante.fit(x_analise_discriminante, y_analise_discriminante)

LinearDiscriminantAnalysis()

In [144]:
dados_analise_disc = {'variavel': analise_discriminante.feature_names_in_, 'coeficiente' : analise_discriminante.coef_[0].round(5)}
pesos_analise_disc = pd.DataFrame(dados_analise_disc).sort_values('variavel').reset_index().drop(columns=['index'])

In [145]:
pesos_analise_disc

,variavel,coeficiente
0,class_contribuinte_nome_BOM PAGADOR,-0.49070
1,class_contribuinte_nome_MELHOR PAGADOR,14.31889
2,class_contribuinte_nome_PAGADOR INTERMEDIARIO,-1.31495
3,class_contribuinte_nome_PIOR PAGADOR,-1.59910
4,class_contribuinte_nome_PRIMEIRA DIVIDA,4.70957


Conclusão: O modelo pode se utilizar do fator de multiplicação do rating do contribuinte para identificar O melhor pagador, o pior pagador, e a primeira dívida. Os demais são intermediários e terão o mesmo peso de atribuição. 

**Como fica a modelagem da regressão**

Variável target:
- % pago da dívida ativa

Variáveis explicativas: 
- Rating do contribuinte baseado no Cluster encontrado
- Idade da dívida em anos
- % De pagamento dos processos de parcelamento (em valor)
- Quantidade de reparcelamentos
- Tipo de dívida

# 06) Salva o modelo de classificação dos contribuintes

In [146]:
def salva_modelo_serializado(nome_modelo_serializado, modelo):
    sav_best_model = open(nome_modelo_serializado, 'wb')
    pickle.dump(modelo, sav_best_model)
    sav_best_model.close()

    #os.makedirs(modelsPath) # nao entendi e tive que colocar isso pra abrir um novo diretorio
    pathModelo = modelsPath+"\\"+os.path.join(nome_modelo_serializado)
    shutil.move(os.path.abspath(nome_modelo_serializado), pathModelo)

In [147]:
salva_modelo_serializado("classificador-contribuinte-v2.pkl", model_predict_contribuinte)